### Load the data for the first time

In [2]:
## fill nas with median
## very rude transformation of dates
## test/train split
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
import xgboost as xgb
import matplotlib
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
# import missingno as msno
import pickle
from sklearn import preprocessing
from sklearn.metrics import roc_curve
# from fancyimpute import *

%matplotlib inline

In [3]:
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
def display_all(df):
    with pd.option_context("display.max_rows", 1000):
        with pd.option_context("display.max_columns", 1000):
            display(df)

def batch_save(train_x, train_y, valid_x, valid_y, test, postfix):
    train_x.reset_index().to_feather("tmp/train_x_{}".format(postfix))
    train_y.reset_index().to_feather("tmp/train_y_{}".format(postfix))
    valid_x.reset_index().to_feather("tmp/valid_x_{}".format(postfix))
    valid_y.reset_index().to_feather("tmp/valid_y_{}".format(postfix))
    test.reset_index().to_feather("tmp/test_{}".format(postfix))
    
def batch_load(postfix):
    train_x = pd.read_feather("tmp/train_x_{}".format(postfix))
    train_y = pd.read_feather("tmp/train_y_{}".format(postfix))
    valid_x = pd.read_feather("tmp/valid_x_{}".format(postfix))
    valid_y = pd.read_feather("tmp/valid_y_{}".format(postfix))
    return train_x, train_y, valid_x, valid_y

def my_roc(y_true, y_prob):
    if isinstance(y_true,pd.core.series.Series):
        y_true = np.array(y_true.tolist())
    if isinstance(y_true,list):
        y_true = np.array(y_true)
    sort_index = np.argsort(y_prob)[::-1]
    y_prob = y_prob[sort_index]
    y_true = y_true[sort_index]
    num_p = y_true.sum()
    num_n = len(y_true) - num_p
    fp = 0
    tp = 0
    fps = []
    tps = []
    prob_prev = -99
    i = 0
    while i < len(y_true):
        if y_prob[i]!=prob_prev:
            fps.append(fp/num_n)
            tps.append(tp/num_p)
            prob_prev=y_prob[i]
        if y_true[i]==1:
            tp+=1
        else:
            fp+=1
        i+=1
    fps.append(fp/num_n)
    tps.append(tp/num_p)
    return np.array(fps), np.array(tps)

# def find_point(t, tpr, fpr):
#     if (fpr==t).argmax()!=0:
#         return tpr[(fpr>=t).argmax()]
#     else:
#         y_2 = tpr[(fpr>t).argmax()]
#         y_1 = tpr[(fpr>t).argmax()-1]
#         x_2 = fpr[(fpr>t).argmax()]
#         x_1 = fpr[(fpr>t).argmax()-1]
#         return ((y_2-y_1)*0.01-x_1*y_2+x_2*y_1)/(x_2-x_1)

# def my_score1(y_prob, xtrain): ##My own version
#     y_true = xtrain.get_label()
#     fpr, tpr = roc_curve(y_true,y_prob)
    
#     # plt.scatter(fpr, tpr)
#     # plt.show()
    
#     return 'score', (0.4*find_point(0.001, tpr, fpr)+0.3*find_point(0.005, tpr, fpr)+0.3*find_point(0.01, tpr, fpr))

# def my_score2(y_prob, y_true): ##My own version
#     fpr, tpr = my_roc(y_true,y_prob)
    
#     plt.scatter(fpr, tpr)
#     plt.show()
    
#     return 'score', (0.4*find_point(0.001, tpr, fpr)+0.3*find_point(0.005, tpr, fpr)+0.3*find_point(0.01, tpr, fpr))

def my_score3(predictions, xtrain): ##Adapted from SKlearn, conservative (actual should be higher)
    ground_truth = y_true = xtrain.get_label()
    fpr,tpr = my_roc(ground_truth, predictions)
#     plt.scatter(fpr, tpr)
#     plt.show()
    tpr1 = tpr[(fpr>=0.001).argmax()-1]
    tpr2 = tpr[(fpr>=0.005).argmax()-1] 
    tpr3 = tpr[(fpr>=0.01).argmax()-1]
    return 'score', 0.4 * tpr1 + 0.3 * tpr2 + 0.3* tpr3

In [4]:
dtype = load_obj('dict_dtype')

my_dict = load_obj('my_dict')

In [ ]:
# df_whole = load_obj('df_whole')

In [ ]:
# array_before_filled = load_obj('array_before_filled')

# filled = MICE().complete(array_before_filled)

In [ ]:
# data = pd.read_csv("atec_anti_fraud_train.csv",parse_dates=['date'])
# test = pd.read_csv("atec_anti_fraud_test_a.csv",parse_dates=['date'])
# df_whole = pd.concat([data.iloc[:,3:], test.iloc[:,2:]])
# info = df_whole.describe()
# dtype = dict()
# null_cols = df_whole.columns[df_whole.isnull().any()].tolist()
# for i in range(1,298):
#     colName = 'f'+str(i)
#     if info[colName][7]<=64:
#         if colName not in null_cols:
#             dtype[colName] = np.int8
#         else:
#             dtype[colName] = np.float16
#     elif info[colName][7]<=32767:
#         if colName not in null_cols:
#             dtype[colName] = np.int16
#         else:
#             dtype[colName] = np.float32
#     else:
#         if colName not in null_cols:
#             dtype[colName] = np.int32
#         else:
#             dtype[colName] = np.float64
# save_obj(dtype, 'dict_dtype')

In [6]:
data = pd.read_csv("atec_anti_fraud_train.csv",parse_dates=['date'], dtype = dtype)

test = pd.read_csv("atec_anti_fraud_test_a.csv",parse_dates=['date'], dtype = dtype)

In [11]:
# for col in data.columns[3:]:
#     print('=======>   '+col+ '   <======')
#     print(data[col].unique())

In [10]:
np.intersect1d(data.id.values, test.id.values)

array([], dtype=object)

In [ ]:
# temporially ignore the rows without labels
data = data[data['label']!=-1]

data.sort_values('date',inplace=True)

In [5]:
df_missing_ratio = load_obj('df_missing_ratio')
display_all(df_missing_ratio)

,positive_missing_ratio,negative_missing_ratio,unlabeled_missing_ratio,test_missing_ratio,labeled_missing_ratio
1,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.487873,0.196766,0.316825,0.250299,0.200330
6,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000
10,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
selected_cols = ['f'+str(item) for item in df_missing_ratio[df_missing_ratio['positive_missing_ratio']<0.1].index.tolist()]
all_nan_cols = ['f'+str(item) for item in df_missing_ratio[df_missing_ratio['positive_missing_ratio']>0.9].index.tolist()]

In [ ]:
# def process_dates(df,date):
#     attrs = ['Day', 'Dayofweek', 'Is_month_end', 'Is_month_start']
#     for attr in attrs:
#         df[attr] = getattr(df[date].dt, attr.lower())
#     return df
# data = process_dates(data,'date')
# test = process_dates(test,'date')

In [ ]:
def fix_missing(train, test, col):
    if is_numeric_dtype(train[col]):
        if train[col].isnull().sum():
            train[col+'_na'] = pd.isnull(train[col])
            test[col+'_na'] = pd.isnull(test[col])
            filler = train[col].median()
            train[col] = train[col].fillna(filler)
            test[col] = test[col].fillna(filler)
    return train, test

for col in data.columns:
    data, test = fix_missing(data, test, col)

In [ ]:
# num_train = data.shape[0]
# num_test = test.shape[0]

In [ ]:
# all_y = data[['label']]

# data = data.drop(['label'], axis=1)

# df_whole = pd.concat([data, test])

# df_whole.index = np.array([_ for _ in range(df_whole.shape[0])])

# save_obj(df_whole, 'df_whole')

In [ ]:
# for col in df_whole.columns:
#     avg = df_whole[col].mean()
#     std = df_whole[col].std()
#     if std != 0:
#         df_whole[col] = (df_whole[col]-avg)/std
#     else:
#         print(col)

In [ ]:
# pca = PCA(n_components=50)
# pca.fit(df_whole.values)
# # save the pca model to the disk
# save_obj(pca, 'pca50')
# # load the pca model from the disk
# pca = load_obj('pca50')
# # data = data[data['label']!=-1]
# df_whole_transformed = pd.DataFrame(pca.transform(df_whole.values), columns=['pcaFeature_{}'.format(i) for i in range(1,51)])

In [ ]:
# train = df_whole_transformed.iloc[:num_train]
# test = df_whole_transformed.iloc[num_train:]
# train_x = train[data[['label']].values!=-1].iloc[:len(data) * 9 // 10]
# valid_x = train[data[['label']].values!=-1].iloc[len(data) * 9 // 10:]
# train_y = data[data[['label']].values!=-1][['label']].iloc[:len(data) * 9 // 10]
# valid_y = data[data[['label']].values!=-1][['label']].iloc[len(data) * 9 // 10:]

In [ ]:
# train_y = data[['label']]
# train_x = data.iloc[:,3:]
# train_x.reset_index().to_feather("tmp/train_x_{}".format('whole_native'))
# train_y.reset_index().to_feather("tmp/train_y_{}".format('whole_native'))

In [ ]:
selected_cols = []
for col in test.columns[2:]:
    if col not in all_nan_cols:
        selected_cols.append(col)

In [ ]:
# use the columns with no or few missing values
data = data[['label']+selected_cols]
test = test[selected_cols]

In [ ]:
train = data.iloc[:len(data) * 9 // 10]
valid = data.iloc[len(data) * 9 // 10:]

In [ ]:
# train.drop('date',axis = 1, inplace=True)
# valid.drop('date',axis = 1, inplace=True)

# features = list(train.columns)[2:]

train_y = train[['label']]
train_x = train.iloc[:,1:]
valid_y = valid[['label']]
valid_x = valid.iloc[:,1:]

In [ ]:
batch_save(train_x, train_y, valid_x, valid_y, test, 'pca50')

### Load the preprocessed data from local files

In [ ]:
train_x, train_y, valid_x, valid_y = batch_load("native")
# print(train_y.label.sum(), train_y.shape)
# print(valid_y.label.sum(), valid_y.shape)

### Define score function

### Baseline prediction with xgboost

In [ ]:
# xgval_1 = xgb.DMatrix(valid_x.iloc[:valid_x.shape[0] // 2,1:].values)
# xgval_2 = xgb.DMatrix(valid_x.iloc[valid_x.shape[0] // 2:,1:].values)

# label_1 = valid_y.iloc[:valid_x.shape[0] // 2,:].iloc[:,1:].values
# label_2 = valid_y.iloc[valid_x.shape[0] // 2:,:].iloc[:,1:].values

# # load the model
# xgb_model = xgb.Booster({'nthread': 4})  # init model
# xgb_model.load_model('model_log/0006.model')  # load model
# print(my_score2(xgb_model.predict(xgval_2), label_2))

In [ ]:
xgtrain = xgb.DMatrix(train_x.iloc[train_x.shape[0]//2:,:].values, label=train_y.iloc[train_y.shape[0]//2:,:].values)
xgval_1 = xgb.DMatrix(valid_x.iloc[:valid_x.shape[0] // 2,:].values,\
                      label=valid_y.iloc[:valid_x.shape[0] // 2,:].values)
xgval_2 = xgb.DMatrix(valid_x.iloc[valid_x.shape[0] // 2:,:].values,\
                      label=valid_y.iloc[valid_x.shape[0] // 2:,:].values)

In [ ]:
evallist = [(xgtrain, 'train'), (xgval_1, 'val_1'), (xgval_2, 'val_2')]

In [ ]:
# set up the parameters
params = {'max_depth': 8, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic'}
params['nthread'] = 4
params['eval_metric'] = ['auc', "error"]
params["scale_pos_weight"] = 5
num_rounds = 20
early_stopping_rounds = 1000

# set up the random seed for testing
# params["seed"] = 6

In [ ]:
%time xgb_model = xgb.train(params, xgtrain, num_rounds, evallist,\
                            feval=my_score3, early_stopping_rounds=early_stopping_rounds,\
                            xgb_model = xgb_model)

In [ ]:
# train the whole dataset (including training set and validation set)
postfix = 'whole_native'
train_x = pd.read_feather("tmp/train_x_{}".format(postfix))
train_y = pd.read_feather("tmp/train_y_{}".format(postfix))
postfix = 'native'
valid_x = pd.read_feather("tmp/valid_x_{}".format(postfix))
valid_y = pd.read_feather("tmp/valid_y_{}".format(postfix))
xgtrain = xgb.DMatrix(train_x.iloc[:,1:].values, label=train_y.label.tolist())
xgval_1 = xgb.DMatrix(valid_x.iloc[:valid_x.shape[0] // 2,1:].values,\
                      label=valid_y.iloc[:valid_x.shape[0] // 2,1:].label.tolist())
xgval_2 = xgb.DMatrix(valid_x.iloc[valid_x.shape[0] // 2:,1:].values,\
                      label=valid_y.iloc[valid_x.shape[0] // 2:,1:].label.tolist())
evallist = [(xgtrain, 'train'), (xgval_1, 'val_1'), (xgval_2, 'val_2')]
xgb_model = xgb.train(params, xgtrain, num_rounds, evallist, feval=my_score1, early_stopping_rounds=early_stopping_rounds)

In [ ]:
# Simple grid search

# max_depths = []
# etas = []
# scale_pos_weights = []
# no_rounds = []

# for max_depth in max_depths:
#     for eta in etas:
#         for scale_pos_weight in scale_pos_weights:
#             for no_round in no_rounds:
#                 params = {'max_depth': max_depth, 'eta': eta, 'silent': 1, 'objective': 'binary:logistic'}
#                 params['nthread'] = 4
#                 params['eval_metric'] = ['auc', "error"]
#                 params["scale_pos_weight"] = scale_pos_weight
#                 num_rounds = no_round
#                 early_stopping_rounds = 120
#                 # set up the random seed for testing
#                 params["seed"] = 6
#                 xgb_model = xgb.train(params, xgtrain, num_rounds, evallist, feval=my_score1,\
#                                       early_stopping_rounds=early_stopping_rounds)

In [ ]:
# save the model
xgb_model.save_model('model_log/0009.model')
# dump model with feature map
xgb_model.dump_model('model_log/dumpraw0009.txt')

In [ ]:
# load the model
xgb_model = xgb.Booster({'nthread': 4})  # init model
xgb_model.load_model('model_log/0007.model')  # load model

In [ ]:
# load test data
test_whole = pd.read_feather("tmp/test_native")

In [ ]:
xgtest = xgb.DMatrix(test.iloc[:,:].values)

# make predictions
preds = xgb_model.predict(xgtest)#, ntree_limit=xgb_model.best_ntree_limit)

res = pd.concat([test_whole.id, pd.Series(list(preds), name='score')], axis=1)

res.to_csv("0007.csv", index=False)

In [ ]:
xgb_model.feature_names

In [ ]:
?xgb.XGBClassifier

### Feature importance

In [ ]:
fig, ax = plt.subplots(figsize=(12,50))
xgb.plot_importance(xgb_model, max_num_features=297, height=0.8, ax=ax)
plt.show()

### Try to fill in na values

In [ ]:
# df_whole = pd.concat([train_x.iloc[:,1:], valid_x.iloc[:,1:], test.iloc[:,2:]])

# msno.matrix(df_whole[df_whole.columns[df_whole.isnull().any()].tolist()])

In [ ]:
# missing columns analysis for training data

data_positive = data[data['label']==1]
data_negative = data[data['label']==0]
data_unlabeled = data[data['label']==-1]

df_missing_ratio = pd.DataFrame(columns=['positive_missing_ratio', 'negative_missing_ratio',\
                                        'unlabeled_missing_ratio', 'test_missing_ratio'])

for col in data_positive.columns[3:]:
    ratio_1 = data_positive[col].isnull().sum() / data_positive[col].shape[0]
    ratio_2 = data_negative[col].isnull().sum() / data_negative[col].shape[0]
    ratio_3 = data_unlabeled[col].isnull().sum() / data_unlabeled[col].shape[0]
    ratio_4 = test[col].isnull().sum() / test[col].shape[0]
    df_missing_ratio = pd.concat([df_missing_ratio,\
                                  pd.DataFrame(np.array([ratio_1, ratio_2, ratio_3, ratio_4]).reshape((1,4)),\
                                               columns = ['positive_missing_ratio', 'negative_missing_ratio',\
                                                          'unlabeled_missing_ratio', 'test_missing_ratio'])])

df_missing_ratio.index = np.arange(1, df_missing_ratio.index.shape[0]+1)

df_missing_ratio['labeled_missing_ratio'] = \
    (df_missing_ratio['positive_missing_ratio']*data_positive.shape[0]+df_missing_ratio['negative_missing_ratio']*data_negative.shape[0])/\
    (data_positive.shape[0]+data_negative.shape[0])

# save_obj(df_missing_ratio, 'df_missing_ratio')

display_all(df_missing_ratio)

# my_dict = dict()
# my_dict['no_missing_cols'] = selected_cols
# my_dict['num_train'] = num_train
# my_dict['num_test'] = num_test
# save_obj(my_dict, 'my_dict')